# Import necessary libraries

Don't forget to install the requirements.txt

In [2]:
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

# Set up your Twitter account credentials and the topic you want to scrape

In [11]:
Acc_Email = 'your_email@example.com'
Acc_Username = 'your_twitter_username'
Acc_Password = 'your_twitter_password'

Tweets_Query = 'pilpres 2024'  # Change this to your desired topic

## Function to parse tweet

In [9]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element(By.XPATH, './/span').text
    try:
        handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    text = card.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
    reply_cnt = card.find_element(By.XPATH, './/div[@data-testid="reply"]').text
    retweet_cnt = card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text
    like_cnt = card.find_element(By.XPATH, './/div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet 

## Login to twitter

In [13]:
# Open a web browser and log in to Twitter
driver = webdriver.Edge() # Use the appropriate driver for your browser
driver.get('https://twitter.com/login')

# Login to twitter
email_input = WebDriverWait(driver, timeout=10).until(
    EC.visibility_of_element_located((By.XPATH, '//input'))
)
email_input.send_keys(Acc_Email)

next_btn = driver.find_element(By.XPATH, '//*[text()="Next"]')
next_btn.click()


# ''' Only use this if asking for unusual activity notification'''
# username_input = WebDriverWait(driver, timeout=10).until(
#     EC.visibility_of_element_located((By.XPATH, '//input'))
# )
# username_input.send_keys(Acc_Username)
# next_btn = driver.find_element(By.XPATH, '//*[text()="Next"]')
# next_btn.click()


# Wait for the password input element to be visible
password_input = WebDriverWait(driver, timeout=10).until(
    EC.visibility_of_element_located((By.XPATH, '//input[@name="password"]'))
)
password_input.send_keys(Acc_Password)

login_btn = driver.find_element(By.XPATH, '//*[text()="Log in"]')
login_btn.click()

## Find the topic

In [14]:
# Navigate to the topic you want to scrape
explore_btn = driver.find_element(By.XPATH, '//a[@href="/explore"]')
explore_btn.click()

explore_input = WebDriverWait(driver, timeout=10).until(
    EC.visibility_of_element_located((By.XPATH, '//input'))
)
explore_input.send_keys(Tweets_Query)
explore_input.send_keys(Keys.ENTER)


latest_btn = driver.find_element(By.XPATH, '//div[@data-testid][2]/div/div[2]')
latest_btn.click()

### Prepare the variables for the data

In [15]:
data = []
tweet_ids = set()

### Scraping the data

You can keep re-running this block of code after changing the query manually to get even more result

In [16]:
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
while scrolling:
    page_cards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(3)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# Save the scraped tweets to a CSV file

In [17]:
# Ensure that the "rawDataset" folder exists
folder_name = 'rawDataset'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

with open(os.path.join(folder_name, 'tweets_dataset.csv'), 'a', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)